In [1]:
import numpy as np
import pandas as pd

In [2]:
#读取数据
data=pd.read_json('ratings.json')
data

,John Carson,Michelle Peterson,William Reynolds,Jillian Hobart,Melissa Jones,Alex Roberts,Michael Henry
Inception,2.5,3.0,2.5,NaN,3,3.0,NaN
Pulp Fiction,3.5,3.5,3.0,3.5,4,4.0,4.5
Anger Management,3.0,1.5,NaN,3.0,2,NaN,NaN
Fracture,3.5,5.0,3.5,4.0,3,5.0,4.0
Serendipity,2.5,3.5,NaN,2.5,2,3.5,1.0
Jerry Maguire,3.0,3.0,4.0,4.5,3,3.0,NaN


## 使用相关系数矩阵作为相似度矩阵

In [3]:
simmat=data.corr()
simmat

,John Carson,Michelle Peterson,William Reynolds,Jillian Hobart,Melissa Jones,Alex Roberts,Michael Henry
John Carson,1.000000,0.396059,0.404520,0.566947,0.594089,0.747018,0.991241
Michelle Peterson,0.396059,1.000000,0.204598,0.314970,0.411765,0.963796,0.381246
William Reynolds,0.404520,0.204598,1.000000,1.000000,-0.258199,0.134840,-1.000000
Jillian Hobart,0.566947,0.314970,1.000000,1.000000,0.566947,0.028571,0.893405
Melissa Jones,0.594089,0.411765,-0.258199,0.566947,1.000000,0.211289,0.924473
Alex Roberts,0.747018,0.963796,0.134840,0.028571,0.211289,1.000000,0.662849
Michael Henry,0.991241,0.381246,-1.000000,0.893405,0.924473,0.662849,1.000000


## 推荐引擎的召回与排序过程

In [16]:
cuser='Michael Henry'
# 召回:从总电影集中找到可能喜欢的电影
#找到相似用户
sim_scores=simmat[cuser]
sim_scores=sim_scores.drop(cuser)
sim_scores=sim_scores[sim_scores>0]
# 遍历每个相似用户看过的电影，做推荐
movie_list={}
for sim_user,sim_score in sim_scores.items():
    movie_scores=data[sim_user].dropna()
    for movie,score in movie_scores.items():
        if pd.isna(data[cuser][movie]):  #cuser没看过
            if movie not in movie_list.keys():
                movie_list[movie] = [[],[]]
            movie_list[movie][0].append(score)
            movie_list[movie][1].append(sim_score)

# 排序:为每一步推荐电影，设计算法求推荐度，并排序
movies=sorted(movie_list.items(),key=lambda x:np.average(x[1][0],weights=x[1][1]),reverse=True)
movies

[('Jerry Maguire',
  [[3.0, 3.0, 4.5, 3, 3.0],
   [0.9912407071619304,
    0.3812464258315117,
    0.8934051474415644,
    0.924473451641905,
    0.6628489803598702]]),
 ('Inception',
  [[2.5, 3.0, 3, 3.0],
   [0.9912407071619304,
    0.3812464258315117,
    0.924473451641905,
    0.6628489803598702]]),
 ('Anger Management',
  [[3.0, 1.5, 3.0, 2],
   [0.9912407071619304,
    0.3812464258315117,
    0.8934051474415644,
    0.924473451641905]])]